In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv('train.csv', index_col=0)
# test = pd.read_csv('test.csv')
# test_labels = pd.read_csv('test_labels.csv')

# split the data in train.csv into training data and testing data
X_train, X_test, y_train, y_test = train_test_split(
     train.iloc[:, 0], train.iloc[:, 1:], test_size=0.2, random_state=123)

In [3]:
pl = Pipeline([
    ('vectorizer', CountVectorizer(binary=True)),
    ('mnb', OneVsRestClassifier(MultinomialNB()))
    ])

pl.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...assifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
          n_jobs=1))])

In [4]:
y_pred = pl.predict_proba(X_test)
roc_auc_score(y_test, y_pred)

0.849343839631958